In [1]:
import pandas as pd
import fastf1 as ff1
import fastf1.plotting
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
import re
import datetime

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_rows = 75
pd.options.display.max_columns = 99

fastf1.Cache.enable_cache('../cache')
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None, misc_mpl_mods=False)

In [2]:
gp_round = 8
url = f'https://f1-jedha-bucket.s3.eu-west-3.amazonaws.com/data/full_data-round_{gp_round}.csv'
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
nb_laps = dict_nb_laps[gp_round]

In [3]:
dffp = pd.read_csv(url, index_col=0)
dffp

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate,Session
1,0 days 00:20:26.667000,1,0 days 00:01:48.574000,2,1,NaN,NaN,0 days 00:00:38.508000,0 days 00:00:44.141000,0 days 00:00:25.925000,0 days 00:19:16.601000,0 days 00:20:00.742000,0 days 00:20:26.667000,194.0,195.0,333.0,313.0,False,MEDIUM,2.0,True,0 days 00:18:38.093000,Red Bull Racing,VER,1,True,2022-06-10 11:03:39.831000,Practice 1
4,0 days 00:27:04.274000,1,0 days 00:01:46.932000,5,1,NaN,NaN,0 days 00:00:37.753000,0 days 00:00:43.278000,0 days 00:00:25.901000,0 days 00:25:55.095000,0 days 00:26:38.373000,0 days 00:27:04.274000,204.0,205.0,334.0,312.0,False,MEDIUM,5.0,True,0 days 00:25:17.342000,Red Bull Racing,VER,2,True,2022-06-10 11:10:19.080000,Practice 1
8,0 days 00:36:46.755000,1,0 days 00:01:46.477000,9,2,NaN,NaN,0 days 00:00:37.746000,0 days 00:00:42.691000,0 days 00:00:26.040000,0 days 00:35:38.024000,0 days 00:36:20.715000,0 days 00:36:46.755000,209.0,216.0,331.0,310.0,False,MEDIUM,10.0,NaN,0 days 00:35:00.278000,Red Bull Racing,VER,1,True,2022-06-10 11:20:02.016000,Practice 1
12,0 days 00:51:01.995000,1,0 days 00:01:45.810000,13,3,NaN,NaN,0 days 00:00:37.090000,0 days 00:00:42.625000,0 days 00:00:26.095000,0 days 00:49:53.275000,0 days 00:50:35.900000,0 days 00:51:01.995000,208.0,209.0,330.0,310.0,True,SOFT,4.0,True,0 days 00:49:16.185000,Red Bull Racing,VER,1,True,2022-06-10 11:34:17.923000,Practice 1
14,0 days 00:55:14.229000,1,0 days 00:01:46.065000,15,3,NaN,NaN,0 days 00:00:37.308000,0 days 00:00:42.668000,0 days 00:00:26.089000,0 days 00:54:05.472000,0 days 00:54:48.140000,0 days 00:55:14.229000,214.0,210.0,330.0,311.0,False,SOFT,6.0,True,0 days 00:53:28.164000,Red Bull Racing,VER,1,True,2022-06-10 11:38:29.902000,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,0 days 01:13:11.148000,63,0 days 00:01:44.573000,11,3,NaN,NaN,0 days 00:00:36.533000,0 days 00:00:42.463000,0 days 00:00:25.577000,0 days 01:12:03.108000,0 days 01:12:45.571000,0 days 01:13:11.148000,223.0,230.0,326.0,307.0,True,SOFT,3.0,True,0 days 01:11:26.575000,Mercedes,RUS,1,True,2022-06-11 11:57:03.832000,Practice 3
333,0 days 00:45:42.416000,77,0 days 00:01:48.149000,3,1,NaN,NaN,0 days 00:00:38.116000,0 days 00:00:44.065000,0 days 00:00:25.968000,0 days 00:44:32.383000,0 days 00:45:16.448000,0 days 00:45:42.416000,222.0,210.0,324.0,308.0,False,SOFT,3.0,True,0 days 00:43:54.267000,Alfa Romeo,BOT,1,True,2022-06-11 11:29:31.524000,Practice 3
335,0 days 00:49:54.401000,77,0 days 00:01:46.947000,5,1,NaN,NaN,0 days 00:00:37.923000,0 days 00:00:43.187000,0 days 00:00:25.837000,0 days 00:48:45.377000,0 days 00:49:28.564000,0 days 00:49:54.401000,218.0,218.0,329.0,310.0,False,SOFT,5.0,True,0 days 00:48:07.454000,Alfa Romeo,BOT,1,True,2022-06-11 11:33:44.711000,Practice 3
337,0 days 00:53:56.488000,77,0 days 00:01:46.600000,7,1,NaN,NaN,0 days 00:00:37.545000,0 days 00:00:43.288000,0 days 00:00:25.767000,0 days 00:52:47.433000,0 days 00:53:30.721000,0 days 00:53:56.488000,215.0,228.0,328.0,310.0,False,SOFT,7.0,True,0 days 00:52:09.888000,Alfa Romeo,BOT,1,True,2022-06-11 11:37:47.145000,Practice 3


In [4]:
dffp = dffp.drop(dffp.columns[[0,5,6,7,8,9,10,11,12,13,14,15,16,17,21,25,26]], axis=1)
dffp

,DriverNumber,LapTime,LapNumber,Stint,Compound,TyreLife,FreshTyre,Team,Driver,TrackStatus,Session
1,1,0 days 00:01:48.574000,2,1,MEDIUM,2.0,True,Red Bull Racing,VER,1,Practice 1
4,1,0 days 00:01:46.932000,5,1,MEDIUM,5.0,True,Red Bull Racing,VER,2,Practice 1
8,1,0 days 00:01:46.477000,9,2,MEDIUM,10.0,NaN,Red Bull Racing,VER,1,Practice 1
12,1,0 days 00:01:45.810000,13,3,SOFT,4.0,True,Red Bull Racing,VER,1,Practice 1
14,1,0 days 00:01:46.065000,15,3,SOFT,6.0,True,Red Bull Racing,VER,1,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...
324,63,0 days 00:01:44.573000,11,3,SOFT,3.0,True,Mercedes,RUS,1,Practice 3
333,77,0 days 00:01:48.149000,3,1,SOFT,3.0,True,Alfa Romeo,BOT,1,Practice 3
335,77,0 days 00:01:46.947000,5,1,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3
337,77,0 days 00:01:46.600000,7,1,SOFT,7.0,True,Alfa Romeo,BOT,1,Practice 3


In [5]:
dffp_temp = dffp.copy()
dffp_temp['LapTime'] = dffp_temp['LapTime'].apply(pd.to_timedelta)
dffp_times = dffp_temp.groupby('Compound')['LapTime'].mean()
dffp_times = pd.DataFrame(dffp_times).sort_values(by='LapTime').reset_index()
dffp_times

,Compound,LapTime
0,SOFT,0 days 00:01:46.738302564
1,HARD,0 days 00:01:48.294769230
2,MEDIUM,0 days 00:01:48.716961748


In [6]:
hard_to_soft = (dffp_times['LapTime'].iloc[2] - dffp_times['LapTime'].iloc[0]) * 0.75
hard_to_soft

Timedelta('0 days 00:00:01.483994388')

In [7]:
medium_to_soft = (dffp_times['LapTime'].iloc[1] - dffp_times['LapTime'].iloc[0]) * 0.75
medium_to_soft

Timedelta('0 days 00:00:01.167349999')

In [8]:
dffp = dffp[dffp['TyreLife'].notna()]
dffp

,DriverNumber,LapTime,LapNumber,Stint,Compound,TyreLife,FreshTyre,Team,Driver,TrackStatus,Session
1,1,0 days 00:01:48.574000,2,1,MEDIUM,2.0,True,Red Bull Racing,VER,1,Practice 1
4,1,0 days 00:01:46.932000,5,1,MEDIUM,5.0,True,Red Bull Racing,VER,2,Practice 1
8,1,0 days 00:01:46.477000,9,2,MEDIUM,10.0,NaN,Red Bull Racing,VER,1,Practice 1
12,1,0 days 00:01:45.810000,13,3,SOFT,4.0,True,Red Bull Racing,VER,1,Practice 1
14,1,0 days 00:01:46.065000,15,3,SOFT,6.0,True,Red Bull Racing,VER,1,Practice 1
...,...,...,...,...,...,...,...,...,...,...,...
324,63,0 days 00:01:44.573000,11,3,SOFT,3.0,True,Mercedes,RUS,1,Practice 3
333,77,0 days 00:01:48.149000,3,1,SOFT,3.0,True,Alfa Romeo,BOT,1,Practice 3
335,77,0 days 00:01:46.947000,5,1,SOFT,5.0,True,Alfa Romeo,BOT,1,Practice 3
337,77,0 days 00:01:46.600000,7,1,SOFT,7.0,True,Alfa Romeo,BOT,1,Practice 3


In [9]:
dffp['LapTime'] = dffp['LapTime'].apply(pd.to_timedelta)

/tmp/ipykernel_1097705/467148075.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffp['LapTime'] = dffp['LapTime'].apply(pd.to_timedelta)


In [10]:
def time_to_seconds(string):
    array = re.findall(r'[0-9]+', str(string))
    array = array[1:]
    array[0] = int(array[0]) * 3600
    array[1] = int(array[1]) * 60
    try : 
        array = float(str(array[0] + array[1] + int(array[2])) + "." + array[3])
        return array
    except :
        return float(str(int(array[-2]) + int(array[-1])) + ".0")

dffp["LapTimeSeconds"] = dffp["LapTime"].apply(time_to_seconds)

/tmp/ipykernel_1097705/3045874443.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffp["LapTimeSeconds"] = dffp["LapTime"].apply(time_to_seconds)


In [11]:
compound_colors = {
    'SOFT': '#FF3333',
    'MEDIUM': '#FFF200',
    'HARD': '#EBEBEB',
}

In [12]:
px.scatter(dffp, 
            x=dffp['TyreLife'] , 
            y='LapTimeSeconds', 
            color='Compound',
            color_discrete_map=compound_colors, 
            width=1000, height=700,
            template='plotly_dark',
            trendline="lowess",
            )

In [13]:
dffp_full = dffp.groupby(['TyreLife','Compound'] ,as_index=False)[['LapTimeSeconds']].mean()
dffp_full = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_full

,TyreLife,Compound,LapTimeSeconds
0,2.0,HARD,109.465000
3,3.0,HARD,109.087333
6,4.0,HARD,106.933000
9,5.0,HARD,106.986000
12,6.0,HARD,107.132250
15,7.0,HARD,107.001000
18,8.0,HARD,107.019500
23,10.0,HARD,108.116000
26,11.0,HARD,108.770000
31,13.0,HARD,109.228000


In [14]:
dffp_filtered = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_filtered = dffp_filtered[dffp_filtered['TyreLife'] > 5.0]
dffp_filtered

,TyreLife,Compound,LapTimeSeconds
12,6.0,HARD,107.132250
15,7.0,HARD,107.001000
18,8.0,HARD,107.019500
23,10.0,HARD,108.116000
26,11.0,HARD,108.770000
31,13.0,HARD,109.228000
34,14.0,HARD,110.193667
37,15.0,HARD,109.168000
40,16.0,HARD,109.736000
13,6.0,MEDIUM,108.429125


In [15]:
dffp_soft = dffp_filtered[dffp_filtered['Compound'] == 'SOFT'].reset_index(drop=True)
dffp_medium = dffp_filtered[dffp_filtered['Compound'] == 'MEDIUM'].reset_index(drop=True)
dffp_hard = dffp_filtered[dffp_filtered['Compound'] == 'HARD'].reset_index(drop=True)

In [16]:
dffp_soft['LapTimeSeconds'].min()

105.49070588235294

In [17]:
dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0]

3.8902941176470733

In [18]:
diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0])
diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0])
diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]

In [19]:
if 'WET' in dffp_filtered['Compound'].tolist() or 'INTERDMEDIATE' in dffp_filtered['Compound'].tolist() or diff_soft < 0 or diff_medium < 0 or diff_hard < 0:
    diff_soft_per_lap = 0.25
    diff_medium_per_lap = 0.15
    diff_hard_per_lap = 0.09
    
else:
    diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0])
    diff_soft_per_lap = diff_soft / len(dffp_soft)
    diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0])
    diff_medium_per_lap = diff_medium / len(dffp_medium)
    diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]
    diff_hard_per_lap = diff_hard / len(dffp_hard)

In [20]:
# if diff_soft > 0 and diff_medium > 0 and diff_hard > 0 :
#     diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0])
#     diff_soft_per_lap = diff_soft / len(dffp_soft)
#     diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0])
#     diff_medium_per_lap = diff_medium / len(dffp_medium)
#     diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]
#     diff_hard_per_lap = diff_hard / len(dffp_hard)

# elif diff_medium > 0:
#     diff_medium_per_lap = diff_medium / len(dffp_medium)
#     diff_soft_per_lap = diff_medium_per_lap * 1.35
#     diff_hard_per_lap = diff_medium_per_lap / 1.35
    
# else:
#     diff_soft_per_lap = 0.25
#     diff_medium_per_lap = 0.15
#     diff_hard_per_lap = 0.09
    

In [21]:
print(diff_soft_per_lap)
print(diff_medium_per_lap)
print(diff_hard_per_lap)

0.38902941176470734
0.09722115384615375
0.28930555555555615


In [22]:
fuel_per_lap_kg = 105 / nb_laps
mean_laps = 60
time_gain_per_kg = 0.035
laps_coeff = mean_laps / nb_laps
time_gained_per_lap = laps_coeff * time_gain_per_kg * fuel_per_lap_kg
time_gained_per_lap

0.08477508650519032

In [23]:
dffp_soft['FuelGain'] = time_gained_per_lap
dffp_soft['FuelGain'] = dffp_soft['FuelGain'].cumsum()
dffp_soft['AdjustedTime'] = dffp_soft['LapTimeSeconds'] + dffp_soft['FuelGain']

In [24]:
laptime_min_series = dffp_full[dffp_full['Compound'] == 'SOFT'].sort_values('LapTimeSeconds', ascending=True).iloc[0]
laptime_min = {'TyreLife': laptime_min_series['TyreLife'], 'LapTimeSeconds': laptime_min_series['LapTimeSeconds']}
laptime_min

{'TyreLife': 6.0, 'LapTimeSeconds': 105.49070588235294}

In [25]:
laptime_min_calc_soft =  (laptime_min['LapTimeSeconds'] - ((laptime_min['TyreLife'] - 1) * diff_soft_per_lap)) + (nb_laps * time_gained_per_lap)
laptime_min_calc_soft

107.86908823529411

In [26]:
laptime_min_calc_medium = (laptime_min_calc_soft + time_to_seconds(medium_to_soft))
laptime_min_calc_medium

109.03643823429411

In [27]:
laptime_min_calc_hard = laptime_min_calc_soft + time_to_seconds(hard_to_soft)
laptime_min_calc_hard

109.35308262329411

In [28]:
def get_deg_values(dc, dr):
    list_times = []

    for i in range(1, nb_laps + 1):
        dr = dr * (1 + dc)
        list_times.append(dr)
        
    return list_times

list_times_soft = get_deg_values(0.12, diff_soft_per_lap)
list_times_medium = get_deg_values(0.09, diff_medium_per_lap)
list_times_hard = get_deg_values(0.07, diff_medium_per_lap)

In [29]:
df_times_soft = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])



df_times_soft['Lap'] = list(range(1, nb_laps + 1))
df_times_soft['Tyre'] = 'SOFT'
df_times_soft['LapTimeSeconds'] = laptime_min_calc_soft
df_times_soft['DeltaDeg'] = list_times_soft
df_times_soft['FuelDeg'] = time_gained_per_lap * df_times_soft['Lap']
df_times_soft['AdjustedTime'] = df_times_soft.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_soft['FinalLapTime'] = df_times_soft.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


In [30]:
df_times_medium = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_medium['Lap'] = list(range(1, nb_laps + 1))
df_times_medium['Tyre'] = 'MEDIUM'
df_times_medium['LapTimeSeconds'] = laptime_min_calc_medium
df_times_medium['DeltaDeg'] = list_times_medium
df_times_medium['FuelDeg'] = time_gained_per_lap * df_times_medium['Lap']
df_times_medium['AdjustedTime'] = df_times_medium.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_medium['FinalLapTime'] = df_times_medium.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_medium

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,MEDIUM,109.036438,0.105971,0.084775,109.142409,109.057634
1,2,MEDIUM,109.036438,0.115508,0.169550,109.151947,108.982397
2,3,MEDIUM,109.036438,0.125904,0.254325,109.162342,108.908017
3,4,MEDIUM,109.036438,0.137236,0.339100,109.173674,108.834573
4,5,MEDIUM,109.036438,0.149587,0.423875,109.186025,108.762150
5,6,MEDIUM,109.036438,0.163050,0.508651,109.199488,108.690837
6,7,MEDIUM,109.036438,0.177724,0.593426,109.214162,108.620737
7,8,MEDIUM,109.036438,0.193719,0.678201,109.230157,108.551957
8,9,MEDIUM,109.036438,0.211154,0.762976,109.247592,108.484616
9,10,MEDIUM,109.036438,0.230158,0.847751,109.266596,108.418845


In [31]:
df_times_hard = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_hard['Lap'] = list(range(1, nb_laps + 1))
df_times_hard['Tyre'] = 'HARD'
df_times_hard['LapTimeSeconds'] = laptime_min_calc_hard
df_times_hard['DeltaDeg'] = list_times_hard
df_times_hard['FuelDeg'] = time_gained_per_lap * df_times_hard['Lap']
df_times_hard['AdjustedTime'] = df_times_hard.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_hard['FinalLapTime'] = df_times_hard.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_hard

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,HARD,109.353083,0.104027,0.084775,109.457109,109.372334
1,2,HARD,109.353083,0.111308,0.169550,109.464391,109.294841
2,3,HARD,109.353083,0.119100,0.254325,109.472183,109.217857
3,4,HARD,109.353083,0.127437,0.339100,109.480520,109.141419
4,5,HARD,109.353083,0.136358,0.423875,109.489440,109.065565
5,6,HARD,109.353083,0.145903,0.508651,109.498985,108.990335
6,7,HARD,109.353083,0.156116,0.593426,109.509199,108.915773
7,8,HARD,109.353083,0.167044,0.678201,109.520127,108.841926
8,9,HARD,109.353083,0.178737,0.762976,109.531820,108.768844
9,10,HARD,109.353083,0.191249,0.847751,109.544331,108.696580


In [32]:
df_times = pd.concat([df_times_soft, df_times_medium, df_times_hard])
df_times['TimeStr'] = df_times['FinalLapTime'].apply(lambda x: str(datetime.timedelta(seconds=x))[2:-3])
df_times

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime,TimeStr
0,1,SOFT,107.869088,0.435713,0.084775,108.304801,108.220026,01:48.220
1,2,SOFT,107.869088,0.487998,0.169550,108.357087,108.187537,01:48.187
2,3,SOFT,107.869088,0.546558,0.254325,108.415647,108.161321,01:48.161
3,4,SOFT,107.869088,0.612145,0.339100,108.481234,108.142133,01:48.142
4,5,SOFT,107.869088,0.685603,0.423875,108.554691,108.130816,01:48.130
...,...,...,...,...,...,...,...,...
46,47,HARD,109.353083,2.337751,3.984429,111.690834,107.706405,01:47.706
47,48,HARD,109.353083,2.501394,4.069204,111.854477,107.785272,01:47.785
48,49,HARD,109.353083,2.676492,4.153979,112.029574,107.875595,01:47.875
49,50,HARD,109.353083,2.863846,4.238754,112.216929,107.978174,01:47.978


In [33]:
hovertemplate = '<b>Lap:</b> %{x}<br><b>Time:</b> %{customdata}'

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'SOFT']['Lap'],
                         y=df_times[df_times['Tyre'] == 'SOFT']['FinalLapTime'],
                         mode='lines',
                         name='Soft',
                         line_color=compound_colors['SOFT'],
                         customdata=df_times[df_times['Tyre'] == 'SOFT']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'MEDIUM']['Lap'],
                         y=df_times[df_times['Tyre'] == 'MEDIUM']['FinalLapTime'],
                         mode='lines',
                         name='Medium',
                         line_color=compound_colors['MEDIUM'],
                         customdata=df_times[df_times['Tyre'] == 'MEDIUM']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'HARD']['Lap'],
                         y=df_times[df_times['Tyre'] == 'HARD']['FinalLapTime'],
                         mode='lines',
                         name='Hard',
                         line_color=compound_colors['HARD'],
                         customdata=df_times[df_times['Tyre'] == 'HARD']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.update_layout(width=1000, height=700, template='plotly_dark', yaxis_range=[df_times['FinalLapTime'].min() - 2, df_times['LapTimeSeconds'].max() + 10], hovermode='x')
fig.show()

In [34]:
df_times.to_csv(f'../data/prediction_times_round_{gp_round}.csv')